In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from keras.models import Sequential
from keras.layers import Dense
import keras.activations,keras.optimizers,keras.losses

In [ ]:
data=pd.read_csv("/kaggle/input/iot-dataset-for-intrusion-detection-systems-ids/BoTNeTIoT-L01-v2.csv")
print(data.columns)
print(data.describe())
print(data.isna().sum())
print(data.info())
print(len(data.columns))


In [ ]:
for i in data.select_dtypes(include='number').columns.values:
    sn.boxplot(data[i])
    plt.show()

In [ ]:
for i in data.select_dtypes(include='object').columns.values:
    if len(data[i].value_counts()) <=10:
        val=data[i].value_counts().values
        index=data[i].value_counts().index
        plt.pie(val,labels=index,autopct='%1.1f%%')
        plt.title(f'The PIE Chart information of {i} column')
        plt.show()

In [ ]:
for i in data.select_dtypes(include='object').columns.values:
    print(data[i].value_counts())
    print("--------------------------------")


In [ ]:
lab=LabelEncoder()
for i in data.select_dtypes(include='object').columns.values:
    data[i]=lab.fit_transform(data[i])


In [ ]:
x={}
X=[]
for i in data.columns.values:
    data['z-scores']=(data[i]-data[i].mean())/(data[i].std())
    outliers=np.abs(data['z-scores'] > 3).sum()
    x[i]=outliers


In [ ]:
for keys,values in x.items():
    if values>0:
        X.append(keys)
print(x)

In [ ]:
x=[]

thresh=2
for i in data[X].columns.values:
    upper=data[i].mean()+thresh*data[i].std()
    lower=data[i].mean()-thresh*data[i].std()
    data=data[(data[i]>lower)&(data[i]<upper)]

print(len(data))

In [ ]:
corr=data.corr()['Attack']
corr=corr.drop(['Attack','z-scores','label'])
for i in corr.index:
    print(i)
    if corr[i] > 0.25:
        x.append(i)

In [ ]:
X=data[x]
data['Attack']=lab.fit_transform(data['Attack'])
Y=data['Attack']
x_train,x_test,y_train,y_test=train_test_split(X,Y)

lr = LogisticRegression(max_iter=500)
lr.fit(x_train, y_train)
print('The logistic regression: ', lr.score(x_test, y_test))

lgb = LGBMClassifier() 
lgb.fit(x_train, y_train)
print('The LGB', lgb.score(x_test, y_test))

tree = DecisionTreeClassifier(criterion='entropy', max_depth=5)
tree.fit(x_train, y_train)
print('Dtree ', tree.score(x_test,y_test))

In [ ]:
x=data[x]
print(len(x))
y=pd.get_dummies(data['Attack'])
x_trin,x_tst,y_trin,y_tst=train_test_split(x,y)
models=Sequential()
models.add(Dense(units=x.shape[1],input_dim=x.shape[1],activation=keras.activations.relu))
models.add(Dense(units=x.shape[1],activation=keras.activations.softmax))
models.add(Dense(units=x.shape[1],activation=keras.activations.softmax))
models.add(Dense(units=x.shape[1],activation=keras.activations.softmax))
models.add(Dense(units=y.shape[1],activation=keras.activations.softmax))
models.compile(optimizer='adam',loss=keras.losses.categorical_crossentropy,metrics='accuracy')
hist=models.fit(x_trin,y_trin,batch_size=500,epochs=50)
plt.plot(hist.history['accuracy'], label='training accuracy', marker='o', color='red')
plt.plot(hist.history['loss'], label='loss', marker='o', color='darkblue')
plt.title('Training Vs  Validation accuracy with adam optimizer')
plt.legend()
plt.show()